In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
diagnosis_path = 'diagnosis.csv'
discharge_path = 'discharge.csv'
edstays_path = 'edstays.csv'
radiology_path = 'radiology.csv'
triage_path = 'triage.csv'

In [4]:
diagnosis_df = pd.read_csv(diagnosis_path)
discharge_df = pd.read_csv(discharge_path)
edstays_df = pd.read_csv(edstays_path)
radiology_df = pd.read_csv(radiology_path)
triage_df = pd.read_csv(triage_path)

# Access information from data frames

**Keeping all the accessed information from the 3 different data frameinto a single data frame (df)**

In [5]:
diagnosis_df.columns

Index(['subject_id', 'stay_id', 'seq_num', 'icd_code', 'icd_version',
       'icd_title'],
      dtype='object')

In [6]:
discharge_df.columns

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')

In [7]:
edstays_df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'gender',
       'race', 'arrival_transport', 'disposition'],
      dtype='object')

In [8]:
radiology_df.columns

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')

In [9]:
triage_df.columns

Index(['subject_id', 'stay_id', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint'],
      dtype='object')

In [10]:
# Drop duplicates based on 'subject_id' and 'hadm_id', keeping the first occurrence
discharge_df_unique = discharge_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')
radiology_df_unique = radiology_df.drop_duplicates(subset=['subject_id', 'hadm_id'], keep='first')

# Rename 'text' column in both dataframes for clarity
discharge_df_unique = discharge_df_unique.rename(columns={'text': 'discharge_notes'})
radiology_df_unique = radiology_df_unique.rename(columns={'text': 'radiology_notes'})

# Merge the dataframes on 'subject_id' and 'hadm_id', ensuring unique combinations
merged_df = pd.merge(discharge_df_unique[['subject_id', 'hadm_id', 'discharge_notes']],
                     radiology_df_unique[['subject_id', 'hadm_id', 'radiology_notes']],
                     on=['subject_id', 'hadm_id'],
                     how='left')



In [11]:
merged_df.head()

,subject_id,hadm_id,discharge_notes,radiology_notes
0,10000032,22595853,\nName: ___ Unit No: _...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000764,27897940,\nName: ___ Unit No: ___\n \...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
2,10002013,24760295,\nName: ___ Unit No: ___\n...,INDICATION: Chest pain.\n\nCOMPARISONS: ___....
3,10002443,21329021,\nName: ___ Unit No: ...,INDICATION: ___ with pericardial tamponade s...
4,10003019,25179393,\nName: ___. Unit No: ___\n \...,INDICATION: Febrile neutropenia. Rule out pn...


In [12]:
merged_df.shape

(14719, 4)

In [13]:
def extract_text_by_subheading(text):
    headings_regex = {
        'EXAMINATION': r'EXAMINATION:(.*?)(?=INDICATION:|$)',
        'INDICATION': r'INDICATION:(.*?)(?=TECHNIQUE:|$)',
        'TECHNIQUE': r'TECHNIQUE:(.*?)(?=COMPARISON:|$)',
        'COMPARISON': r'COMPARISON:(.*?)(?=FINDINGS:|$)',
        'FINDINGS': r'FINDINGS:(.*?)(?=IMPRESSION:|$)',
        'IMPRESSION': r'IMPRESSION:(.*)'
    }
    extracted_text = {}
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(1).strip()
        else:
            extracted_text[heading] = None
    return extracted_text

# Apply the function to each row in the 'radiology_notes' column and create a new DataFrame from the results
extracted_df = pd.DataFrame(merged_df['radiology_notes'].apply(extract_text_by_subheading).tolist())

# Concatenate the new DataFrame with the original merged_df
merged_df_expanded = pd.concat([merged_df, extracted_df], axis=1)
merged_df_expanded.head()

,subject_id,hadm_id,discharge_notes,radiology_notes,EXAMINATION,INDICATION,TECHNIQUE,COMPARISON,FINDINGS,IMPRESSION
0,10000032,22595853,\nName: ___ Unit No: _...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,CHEST (PA AND LAT),___ with new onset ascites // eval for infection,Chest PA and lateral,None.,"There is no focal consolidation, pleural effus...",No acute cardiopulmonary process.
1,10000764,27897940,\nName: ___ Unit No: ___\n \...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,CHEST (PA AND LAT),"___ with hypoxia // ?pna, aspiration.\n\nCOMP...",None,None,PA and lateral views of the chest provided. ...,"Focal consolidation at the left lung base, pos..."
2,10002013,24760295,\nName: ___ Unit No: ___\n...,INDICATION: Chest pain.\n\nCOMPARISONS: ___....,None,Chest pain.\n\nCOMPARISONS: ___.\n\nFINDINGS:...,None,None,A moderate left pleural effusion is new since ...,New moderate left pleural effusion with adjace...
3,10002443,21329021,\nName: ___ Unit No: ...,INDICATION: ___ with pericardial tamponade s...,None,___ with pericardial tamponade status post dra...,Two portable views the chest.,None,There is moderate enlargement of the cardiac s...,Moderate enlargement of the cardiac silhouette...
4,10003019,25179393,\nName: ___. Unit No: ___\n \...,INDICATION: Febrile neutropenia. Rule out pn...,None,Febrile neutropenia. Rule out pneumonia.\n\nC...,Upright PA and lateral radiographs of the ches...,Chest radiographs ___ and ___. CT\nchest ___....,Chain suture projects over the right upper hem...,"Increase, mild atelectasis at the left base bu..."


In [14]:
import pandas as pd
import re

def extract_text_by_subheading_discharge(text):
    
    headings_regex = {
    'Chief Complaint': r'(?:CC:|Chief Complaint:| \n___ Complaint:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Service': r'(?:Service:|Specialty:|Unit:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Major Surgical or Invasive Procedure': r'(?:Major Medical Procedures:|Major Procedures|Major Surgeries|Major Surgical or Invasive Procedure)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'History of Present Illness': r'(?:HPI:|History of Present Illness:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Past Medical History': r'(?:PMH:|Past Medical History:|PAST MEDICAL HISTORY:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Social History': r'(?:Social History:|SOC:|SH:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Family History': r'(?:Family History:|PFH:|FH:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Past Surgical History': r'(?:Past Surgical History:|PSH:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Problem List': r'(?:Problem List:|Problems:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Physical Exam': r'(?:Physical Exam:|PE:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Medication lists': r'(?:Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Pertinent Results': r'(?:Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Brief Hospital Course': r'(?:BHC:|Brief Hospital Course:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Disposition': r'(?:Disposition:|Dispo:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Discharge Diagnosis': r'Discharge Diagnosis:\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Discharge Instructions': r'(?:Discharge Instructions:|Patient Instructions:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Followup Instructions': r'(?:Followup Instructions:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
    'Transitional Issues': r'(?:Transitional Issues:)\s*(.*?)(?=\n[A-Za-z ]+:|$)'
}


    extracted_text = {}
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(1).strip() if match.group(1) else None
        else:
            extracted_text[heading] = None
    return extracted_text

# Apply the function to the 'discharge_notes' column
extracted_discharge = pd.DataFrame(merged_df_expanded['discharge_notes'].apply(extract_text_by_subheading_discharge).tolist())

# Concatenate the new DataFrame with the original merged_df to include the new columns
merged_df_expanded_with_discharge = pd.concat([merged_df_expanded, extracted_discharge], axis=1)
merged_df_expanded_with_discharge.head()

,subject_id,hadm_id,discharge_notes,radiology_notes,EXAMINATION,INDICATION,TECHNIQUE,COMPARISON,FINDINGS,IMPRESSION,Chief Complaint,Service,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,Family History,Past Surgical History,Problem List,Physical Exam,Medication lists,Pertinent Results,Brief Hospital Course,Disposition,Discharge Diagnosis,Discharge Instructions,Followup Instructions,Transitional Issues
0,10000032,22595853,\nName: ___ Unit No: _...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,CHEST (PA AND LAT),___ with new onset ascites // eval for infection,Chest PA and lateral,None.,"There is no focal consolidation, pleural effus...",No acute cardiopulmonary process.,Worsening ABD distension and pain,MEDICINE,:\nParacentesis,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis \n2. No history of abnormal ...,___,"She a total of five siblings, but she is not ...",None,None,VS: 98.1 107/61 78 18 97RA,The Preadmission Medication list is accurate a...,___ 10:25PM GLUCOSE-109* UREA N-25* CREAT-0....,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",Home,Ascites from Portal HTN,"Dear Ms. ___,\nIt was a pleasure taking care o...",___,None
1,10000764,27897940,\nName: ___ Unit No: ___\n \...,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,CHEST (PA AND LAT),"___ with hypoxia // ?pna, aspiration.\n\nCOMP...",None,None,PA and lateral views of the chest provided. ...,"Focal consolidation at the left lung base, pos...",Epistaxis,MEDICINE,:\nNone,Mr. ___ is an ___ with history of AAA s/p repa...,MI after AAA repair when he was ___ y/o\nHTN\n...,___,Patient is unaware of a family history of blee...,None,None,ADMISSION:,The Preadmission Medication list is accurate a...,ADMISSION:\n___ 08:15PM BLOOD WBC-11.3* RBC-4....,Mr. ___ is an ___ with history of AAA s/p repa...,Home With Service,Nasal fracture\nEpistaxis\nNSTEMI,"Mr. ___,\nYou were admitted after you fell and...",___,None
2,10002013,24760295,\nName: ___ Unit No: ___\n...,INDICATION: Chest pain.\n\nCOMPARISONS: ___....,None,Chest pain.\n\nCOMPARISONS: ___.\n\nFINDINGS:...,None,None,A moderate left pleural effusion is new since ...,New moderate left pleural effusion with adjace...,chest pain,MEDICINE,:\ncardiac catheterization,"___ w/ PMH of CAD s/p PCI x3, s/p off-pump CAB...","1. CARDIAC RISK FACTORS: (-)Diabetes, (+)Dysli...",___,She was a ward of the ___ and does not know he...,None,None,Admission:\nVS- T 99.4 BP 157/88 HR 118 RR 24 ...,The Preadmission Medication list is accurate a...,Admission labs:\n___ 03:45PM WBC-8.9 RBC-3.6...,___ year old female with a history of CAD s/p ...,Home,Primary:\nnon-ST elevation MI\ncoronary artery...,It was a pleasure taking care of you during yo...,___,None
3,10002443,21329021,\nName: ___ Unit No: ...,INDICATION: ___ with pericardial tamponade s...,None,___ with pericardial tamponade status post dra...,Two portable views the chest.,None,There is moderate enlargement of the cardiac s...,Moderate enlargement of the cardiac silhouette...,chest pain,MEDICINE,:\n___ - bedside pericardiocentesis at ___,HISTORY OF PRESENTING ILLNESS: Mr. ___ is a __...,PAST MEDICAL HISTORY:,___,paternal history of ureothelial carcinoma.\nMa...,None,None,ADMISSION PHYSICAL EXAMINATION: \n===========...,The Preadmission Medication list is accurate a...,ADMISSION LABS:\n===============\n___ 10:57PM ...,SUMMARY: \n===================== \n___ male wi...,Home,Primary Diagnosis: inflammatory pericarditis,"Dear Mr. ___, \n \nWHY WERE YOU ADMITTED TO T...",___,None
4,10003019,25179393,\nName: ___. Unit No: ___\n \...,INDICATION: Febrile neutropenia. Rule out pn...,None,Febrile neutropenia. Rule out pneumonia.\n\nC...,Upright PA and lateral radiographs of the ches...,Chest radiographs ___ and ___. CT\nchest ___....,Chain suture projects over the right upper hem...,"Increase, mild atelectasis at the left base bu...",Neutropenic Fever,MEDICINE,:\nNone,___ yo male with h/o Stage IV Hodgkin's lympho...,PAST ON

In [18]:
import os
import openai
import pandas as pd

# Set the API key
openai.api_key = os.getenv("OPENAI_API_KEY")

def query_with_chatgpt(question, df, columns, hadm_id):
    # Filter the dataframe for the specific hadm_id
    df = df[df['hadm_id'] == hadm_id]
    
    if df.empty:
        return "No records found for the given HADM ID."
    
    # Create a context string from the specified columns
    context_data = df[columns].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep=' ')
    
    # Limit the context data length if necessary
    max_length = 4000  # Adjust according to the GPT-3 token limit
    if len(context_data) > max_length:
        context_data = context_data[:max_length]
    
    # Create the prompt for the API
    prompt = f"Question: {question}\nData: {context_data}\nAnswer:"
    
    try:
        # Call the OpenAI API
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=150,
            temperature=0.5
        )
        # Extract the text from the response
        answer = response.choices[0].text.strip()
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
question = "What major treatments, surgeries or procedures did the patient receive?"
hadm_id = 22595853  # Specify the HADM ID you want to query
relevant_cols = ['History of Present Illness','Major Surgical or Invasive Procedure']  # Specify columns related to the question
answer = query_with_chatgpt(question, merged_df_expanded_with_discharge, relevant_cols, hadm_id)
print(answer)


The patient received a paracentesis, which is a procedure to remove fluid from the abdominal cavity. She also received blood tests, including liver function tests and a complete blood count, to evaluate her liver function and overall health.
